# Reading dataset

`

In [1]:
import pandas as pd

data = pd.read_csv('stroke-balanced.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,44912,Male,12.0,0,0,No,children,Urban,67.06,16.1,Unknown,0
1,66972,Female,52.0,0,0,Yes,Govt_job,Urban,80.88,23.8,smokes,0
2,1451,Female,17.0,0,0,No,Private,Urban,78.46,23.5,Unknown,0
3,49797,Female,28.0,0,0,No,Private,Rural,75.53,34.9,never smoked,0
4,70241,Female,22.0,0,0,No,Private,Urban,66.29,20.5,smokes,0


# Handling missing values

In [2]:
mask = data.isnull().any(axis=1) 
print(mask)

0      False
1      False
2      False
3      False
4      False
       ...  
991     True
992    False
993    False
994     True
995    False
Length: 996, dtype: bool


In [3]:
data_clean = data[~mask] # remove rows with missing data

# Input and output

In [4]:
data_input = data_clean.drop(columns=['id', 'stroke'])  # Select all columns from `data_clean` except `id` and `stroke`
data_output = data_clean['stroke'] # Select `stroke` column from `data_clean`

# Handling categorical data
## Numeric encoding

In [5]:
# print data types of `data_input`
data_input.dtypes

gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
dtype: object

In [6]:
# print unique values of categorical features
print(data_input['gender'].unique())
print(data_input['ever_married'].unique())
print(data_input['work_type'].unique()) 
print(data_input['Residence_type'].unique())
print(data_input['smoking_status'].unique())

['Male' 'Female']
['No' 'Yes']
['children' 'Govt_job' 'Private' 'Self-employed' 'Never_worked']
['Urban' 'Rural']
['Unknown' 'smokes' 'never smoked' 'formerly smoked']


In [7]:
data_input_encoded_1 = data_input.replace({
    # use numeric encoding to encode `gender`, `ever_married`, and `Residence_type`
    'gender':{'Male':0, 'Female':1},
    'ever_married':{'No':0, 'Yes':1},
    'Residence_type':{'Urban':0, 'Rural':1}
})

In [8]:
data_input_encoded_1.dtypes

gender                 int64
age                  float64
hypertension           int64
heart_disease          int64
ever_married           int64
work_type             object
Residence_type         int64
avg_glucose_level    float64
bmi                  float64
smoking_status        object
dtype: object

## One-hot encoding

In [9]:
data_input_encoded_2 = pd.get_dummies(data_input_encoded_1)  # apply one-hot encoding to data_input_encoded_1

In [10]:
data_input_encoded_2.head()

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,12.0,0,0,0,0,67.06,16.1,0,0,0,0,1,1,0,0,0
1,1,52.0,0,0,1,0,80.88,23.8,1,0,0,0,0,0,0,0,1
2,1,17.0,0,0,0,0,78.46,23.5,0,0,1,0,0,1,0,0,0
3,1,28.0,0,0,0,1,75.53,34.9,0,0,1,0,0,0,0,1,0
4,1,22.0,0,0,0,0,66.29,20.5,0,0,1,0,0,0,0,0,1


# Split into (train - validation - test)

In [11]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split( data_input_encoded_2, data_output, test_size=0.20, random_state = 1 )

X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.25, random_state = 1 )

In [12]:
# print(X_train.shape)
# print(y_train.shape)
# print('---------------------')
# print(X_val.shape)
# print(y_val.shape)
# print('---------------------')
# print(X_test.shape)
# print(y_test.shape)

# Feature scaling (Normalization)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.fit_transform(X_test)
X_val_scaled = sc.fit_transform(X_val)

# naive_bayes
## Training and Validation

In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [15]:
classifier = GaussianNB()
classifier.fit(X_train_scaled, y_train)

GaussianNB()

In [16]:
y_pred_train = classifier.predict(X_train_scaled)
y_pred_val = classifier.predict(X_val_scaled)

print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_val, y_pred_val))

0.5772058823529411
0.532967032967033


#  Logistic Regression


In [17]:
from sklearn.linear_model import LogisticRegression
logistic_clf = LogisticRegression(random_state = 1)
logistic_clf.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
y_pred_train = logistic_clf.predict(X_train_scaled)
y_pred_val = logistic_clf.predict(X_val_scaled)

print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_val, y_pred_val))

0.7941176470588235
0.7802197802197802


## Testing

In [20]:
y_pred_test = logistic_clf.predict(X_test_scaled)
print(accuracy_score(y_test, y_pred_test))

0.7307692307692307
